In [46]:
!curl "https://api.mockaroo.com/api/407bc5e0?count=1000&key=1dfae2c0" > "Employee.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  107k    0  107k    0     0  63964      0 --:--:--  0:00:01 --:--:-- 63953


In [47]:
!curl "https://api.mockaroo.com/api/2b2a8af0?count=1000&key=1dfae2c0" > "Department1.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1029k    0 1029k    0     0   493k      0 --:--:--  0:00:02 --:--:--  494k


In [48]:
import sqlite3
import pandas as pd
import os

In [49]:
employee_schema = """
CREATE TABLE IF NOT EXISTS employee (
	employee_id INT PRIMARY KEY ,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	gender VARCHAR(50),
	date_of_birth DATE,
	job_title VARCHAR(50),
	department VARCHAR(50),
	salary DECIMAL(8,2),
	hire_date DATE,
	email VARCHAR(50)
);
"""
department_schema = """
CREATE TABLE IF NOT EXISTS department (

	department_name VARCHAR(50),
	department_code VARCHAR(10) ,
	department_head VARCHAR(50),
	department_budget VARCHAR(50),
	department_location VARCHAR(50),
	department_phone VARCHAR(50),
	department_email VARCHAR(50),
	department_description TEXT,
	department_creation_date DATE,
	department_website VARCHAR(1000),
   department_id INT PRIMARY KEY
);
"""

In [50]:
db_name ='organization.db'
if os.path.exists(db_name):
  os.remove(db_name)
  print(f"Removed existing database'{db_name}")

Removed existing database'organization.db


In [51]:
import sqlite3
import pandas as pd
import os



COLUMN_DATA_TYPES = {
    'employee' : {
	'employee_id': 'int64' ,
	'first_name': 'object',
	'last_name' : 'object',
	'gender' : 'object',
	'date_of_birth' :'datetime64[ns]',
	'job_title' : 'object',
	'department': 'object',
	'salary' : 'float64',
	'hire_date' : 'datetime64[ns]',
	'email' : 'object'
},

'department' :{

	'department_name' : 'object',
	'department_code' :'object',
	'department_head' : 'object',
	'department_budget' : 'object',
	'department_location' :'object',
	'department_phone': 'object',
	'department_email': 'object',
	'department_description': 'object',
	'department_creation_date': 'datetime64[ns]',
	'department_website': 'object',
 'department_id': 'int64'
}
}

#Database Setup

db_name ='organization.db'
conn = None
try:
  conn = sqlite3.connect(db_name)
  cursor=conn.cursor()
  print(f"Database '{db_name}' created and connected successfully. ✅")
  # Create tables
  cursor.execute(employee_schema)
  cursor.execute(department_schema)
  print("Tables 'employee', 'department' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
  csv_to_table_map = {
        '/content/Department1.csv': 'department',
        '/content/Employee.csv': 'employee',

    }
  for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
  conn.commit()
  print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")


Database 'organization.db' created and connected successfully. ✅
Tables 'employee', 'department' created successfully.

Processing '/content/Department1.csv' for table 'department'...
  -> Data from '/content/Department1.csv' loaded into 'department' table successfully.

Processing '/content/Employee.csv' for table 'employee'...
  -> Data from '/content/Employee.csv' loaded into 'employee' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


In [52]:
from google import genai
from google.colab import userdata

In [53]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [54]:
prompt = """

###ROLE###
You are a highly skilled Text-to-SQL translator with expertise in SQL syntax, database schema interpretation, and natural language understanding. You generate syntactically correct and semantically accurate SQL queries based on user input and a given database schema.

###CONTEXT###
The user is working with a relational database for an organization platform. The database includes three main tables: `employee`, `department`. The goal is to allow users to input natural language queries (in English), and have the model return equivalent SQL statements that accurately extract the requested data using the given schema.

Here is the full schema:
**employee table**
```sql
CREATE TABLE IF NOT EXISTS employee (
	employee_id INT PRIMARY KEY ,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	gender VARCHAR(50),
	date_of_birth DATE,
	job_title VARCHAR(50),
	department VARCHAR(50),
	salary DECIMAL(8,2),
	hire_date DATE,
	email VARCHAR(50)
);
````
**department table **
```sql
CREATE TABLE IF NOT EXISTS department (

	department_name VARCHAR(50),
	department_code VARCHAR(10) ,
	department_head VARCHAR(50),
	department_budget VARCHAR(50),
	department_location VARCHAR(50),
	department_phone VARCHAR(50),
	department_email VARCHAR(50),
	department_description TEXT,
	department_creation_date DATE,
	department_website VARCHAR(1000),
   department_id INT PRIMARY KEY
);
````




###TASK###
Your task is to:

1. Read a natural language query about the e-commerce data.
2. Interpret the user's intent based on the schema provided.
3. Generate a valid SQL `SELECT` query that returns the expected result.
4. Ensure correct table joins, column selection, filtering, and grouping as necessary.
5. Handle aggregate functions (e.g., `COUNT`, `AVG`, `SUM`) where appropriate.
6. Disambiguate user terms based on schema details (e.g., "buyer" → `customers`, "product rating" → `products.rating`, etc.).

###CONSTRAINTS###

* Only return a valid SQL query as output — no explanations or extra text.
* The user is using sqllite database - respond with correct and valid sqllite syntax
* Use aliases (`AS`) for column names only when the original name is ambiguous.
* Do not create or modify tables.
* Do not assume the existence of tables or columns not provided in the schema.
* Avoid subqueries unless absolutely necessary for correctness or performance.
* Prefer readability: indent joins and clauses properly.

###EXAMPLES###
**Input:** "Show me the names and emails of employee in IT department"
**Output:**

```sql
SELECT first_name, last_name, email
FROM employee
where department = 'IT'
```


###OUTPUT FORMAT###
Return only the sqllite SQL query as a code block using triple backticks and the `sql` language tag, like this:

```sql
-- Your SQL query here
```
"""

In [55]:
import json
def get_sql_query_via_gemini(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response) # uncomment this and understand at the output

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = response.text.replace('```sql', '').replace('```', '')

  return output


In [56]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='organization.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [57]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query_via_gemini(genai_client, prompt, user_query)
  results = execute_query(output)
  return results

In [58]:
text2sql(genai_client, prompt, "Show me the employee who are females")

Input Token Count: 736
Thoughts Token Count: 35
Output Token Count: 49
Total Token Count: 820

Executing query on 'organization.db':

SELECT employee_id, first_name, last_name, gender, date_of_birth, job_title, department, salary, hire_date, email
FROM employee
WHERE gender = 'Female'

Query executed successfully.


,employee_id,first_name,last_name,gender,date_of_birth,job_title,department,salary,hire_date,email
0,2,Shoshana,Springham,Female,1994-11-08 00:00:00,Associate Professor,Support,43907.85,2012-05-15 00:00:00,sspringham1@cafepress.com
1,6,Jorry,Aikman,Female,1963-01-19 00:00:00,VP Quality Control,Marketing,54719.17,2013-12-07 00:00:00,jaikman5@w3.org
2,8,Ashlan,Reck,Female,1992-12-01 00:00:00,Quality Control Specialist,Services,115711.35,2018-02-10 00:00:00,areck7@usatoday.com
3,9,Catlaina,Stainton,Female,1989-07-12 00:00:00,Marketing Manager,Sales,113333.02,2018-05-11 00:00:00,cstainton8@amazon.co.uk
4,10,Marielle,Sigge,Female,1982-09-24 00:00:00,Accountant II,Marketing,108214.95,2016-03-31 00:00:00,msigge9@unicef.org
...,...,...,...,...,...,...,...,...,...,...
428,986,Ilsa,Fratson,Female,1968-08-12 00:00:00,General Manager,Services,68073.58,2020-10-04 00:00:00,ifratsonrd@opera.com
429,988,Darlleen,Hidderley,Female,1978-09-19 00:00:00,Quality Engineer,Marketing,83329.39,2018-06-29 00:00:00,dhidderleyrf@creativecommons.org
430,990,Olivette,Faragher,Female,1972-12-14 00:00:00,Community Outreach Specialist,Services,147568.41,2015-06-04 00:00:00,ofaragherrh@uol.com.br
431,994,Lotte,Truwert,Female,1984-11-16 00:00:00,Associate Professor,Training,105736.82,2012-09-15 00:00:00,ltruwertrl@amazon.com


In [60]:
text2sql(genai_client, prompt, "total number of employees with department sales")

Input Token Count: 736
Thoughts Token Count: 64
Output Token Count: 22
Total Token Count: 822

Executing query on 'organization.db':

SELECT COUNT(employee_id)
FROM employee
WHERE department = 'Sales'

Query executed successfully.


,COUNT(employee_id)
0,78
